In [1]:
import cv2
import glob
import numpy as np
import os.path as path
import os
from scipy import misc
#from keras.applications.resnet50 import ResNet50
#from keras.applications.densenet import DenseNet121, decode_predictions
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Input, Dense, Conv2D, Conv1D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation, Lambda
from keras.callbacks import History
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.losses import categorical_crossentropy
from keras.models import Model, Input
from keras import models
from keras import layers
import keras
import numpy as np
import warnings
import matplotlib.pyplot as plt
import pickle
import tensorflow as tf
import pandas as pd
import sklearn.metrics as sm

from keras.layers import BatchNormalization
from keras.layers import Concatenate
from keras.layers import Add

from keras.layers import Conv2DTranspose
from keras import backend as K
from keras.layers import GlobalAveragePooling2D, Reshape, Dense, Permute, multiply

from skimage.transform import resize

os.environ['CUDA_VISIBLE_DEVICES']='0'

Using TensorFlow backend.


In [2]:
# !pip list

In [3]:
img_rows, img_cols, channel = 224, 224, 3 # Resolution of inputs
    
#resnet = ResNet50(input_shape = (img_rows, img_cols, channel))
#densenet = DenseNet121(include_top=False, 
#                     weights='imagenet', 
#                     input_tensor=None, 
#                     input_shape=(img_rows, img_cols, channel), 
#                     pooling=None)

In [4]:
# parameter setting

#img_rows, img_cols = 224, 224 # Resolution of inputs
img_rows, img_cols = 64, 64 # Resolution of inputs
channel = 3
num_classes = 2 
batch_size = 16
nb_epoch = 50

learning_rate = 1e-4

In [5]:
def FireNet(img_rows, img_cols, channel):
    model_input = keras.layers.Input(shape=(img_rows, img_cols, channel))
    
    # entranse convolution part
    entrance_featmap = Conv2D(filters = 16,
                              kernel_size = (3, 3),
                              activation='relu',
                              strides = [1,1],
                              input_shape = (img_rows, img_cols, channel),
                              padding = 'same')(model_input)
    
    pooling_1 = AveragePooling2D(pool_size=(2, 2), strides=[2,2], padding='valid')(entrance_featmap)
    pooling_1_drop = Dropout(0.5)(pooling_1)
    
    x_conv_2 = Conv2D(filters = 32, kernel_size = (3,3), padding = 'same', activation='relu')(pooling_1_drop)
    pooling_2 = AveragePooling2D(pool_size=(2, 2), strides=[2,2], padding='valid')(x_conv_2)
    pooling_2_drop = Dropout(0.5)(pooling_2)
    
    x_conv_3 = Conv2D(filters = 64, kernel_size = (3,3), padding = 'same', activation='relu')(pooling_2_drop)
    pooling_3 = AveragePooling2D(pool_size=(2, 2), strides=[2,2], padding='valid')(x_conv_3)
    pooling_3_drop = Dropout(0.5)(pooling_3)
    
    layer_4 = Flatten()(pooling_3_drop)
    layer_4_dense = Dense(256, activation = 'relu')(layer_4)
    layer_4_dense_drop = Dropout(0.2)(layer_4_dense)
    
    layer_5_dense = Dense(128, activation = 'relu')(layer_4_dense_drop)
    output_ = Dense(2, activation = 'softmax')(layer_4_dense_drop)
    print(output_)
    
    model = keras.models.Model(inputs = model_input, outputs = output_)
    return model

In [6]:
model = FireNet(img_rows, img_cols, channel)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Tensor("dense_3/Softmax:0", shape=(?, 2), dtype=float32)


In [7]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 16)        448       
_________________________________________________________________
average_pooling2d_1 (Average (None, 32, 32, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        4640      
_________________________________________________________________
average_pooling2d_2 (Average (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 16, 32)        0   

In [8]:
from keras import optimizers
from keras.callbacks import LearningRateScheduler
import math

opt_adam = keras.optimizers.Adam(lr=learning_rate)

class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = [1,1]
        self.lr = []

    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.lr.append(step_decay(len(self.losses)))
        
def step_decay(epoch):
    if epoch <= 10:
        lrate = 1e-4
    elif epoch > 10 and epoch <= 20:
        lrate = 1e-5
    elif epoch > 20 and epoch <= 30:
        lrate = 2*1e-6
    elif epoch > 30 and epoch <= 40:
        lrate = 1e-6
    elif epoch > 40:
        lrate = 1e-7
    else:
         print('invalid')
    return lrate

mcp_save = ModelCheckpoint('./ckpt_folder/firenet_still.hdf5', save_best_only=True, monitor='val_accuracy', mode='max')   
history=LossHistory()
lrate=LearningRateScheduler(step_decay)

#lrate=LearningRateScheduler(step_decay)

In [10]:
# load dataset
with open ('./datafiles_aug/x_train_fismo', 'rb') as fp:
    x_train_fismo = pickle.load(fp, encoding='latin1')
with open ('./datafiles_aug/x_train_bow', 'rb') as fp:
    x_train_bow = pickle.load(fp, encoding='latin1')
with open ('./datafiles_aug/x_train_sharma', 'rb') as fp:
    x_train_sharma = pickle.load(fp, encoding='latin1')
with open ('./datafiles_aug/x_test_fismo', 'rb') as fp:
    x_test_fismo = pickle.load(fp, encoding='latin1')
with open ('./datafiles_aug/x_test_bow', 'rb') as fp:
    x_test_bow = pickle.load(fp, encoding='latin1')
with open ('./datafiles_aug/x_test_sharma', 'rb') as fp:
    x_test_sharma = pickle.load(fp, encoding='latin1')

with open ('./datafiles_aug/x_train_still.pickle', 'rb') as fp:
    x_train_still = pickle.load(fp, encoding='latin1') 

with open ('./datafiles_aug/y_train_still.pickle', 'rb') as fp:
    y_train_still = pickle.load(fp, encoding='latin1')   
    
with open ('./datafiles_aug/x_test_still.pickle', 'rb') as fp:
    x_test_still = pickle.load(fp, encoding='latin1') 

with open ('./datafiles_aug/y_test_still.pickle', 'rb') as fp:
    y_test_still = pickle.load(fp, encoding='latin1') 
    
print('load input data clear~!! =========================================')

with open ('./datafiles_aug/y_train_fismo', 'rb') as fp:
    y_train_fismo = pickle.load(fp, encoding='latin1')
with open ('./datafiles_aug/y_test_fismo', 'rb') as fp:
    y_test_fismo = pickle.load(fp, encoding='latin1')
with open ('./datafiles_aug/y_train_bow', 'rb') as fp:
    y_train_bow = pickle.load(fp, encoding='latin1')
with open ('./datafiles_aug/y_test_bow', 'rb') as fp:
    y_test_bow = pickle.load(fp, encoding='latin1')
with open ('./datafiles_aug/y_train_sharma', 'rb') as fp:
    y_train_sharma = pickle.load(fp, encoding='latin1')
with open ('./datafiles_aug/y_test_sharma', 'rb') as fp:
    y_test_sharma = pickle.load(fp, encoding='latin1')

print('load label data clear~!! =========================================')
print('load dataset clear~!! ============================================')
    
print('x_train_fismo: ' + str(np.shape(x_train_fismo)))
print('x_test_fismo: '+ str(np.shape(x_test_fismo)))
print('y_train_fismo: ' + str(np.shape(y_train_fismo)))
print('y_test_fismo: ' + str(np.shape(y_test_fismo)))
print('==================================================================')
print('x_train_bow: ' + str(np.shape(x_train_bow)))
print('x_test_bow: '+ str(np.shape(x_test_bow)))
print('y_train_bow: ' + str(np.shape(y_train_bow)))
print('y_test_bow: ' + str(np.shape(y_test_bow)))
print('==================================================================')
print('x_train_sharma: ' + str(np.shape(x_train_sharma)))
print('x_test_sharma: '+ str(np.shape(x_test_sharma)))
print('y_train_sharma: ' + str(np.shape(y_train_sharma)))
print('y_test_sharma: ' + str(np.shape(y_test_sharma)))

x_train = np.concatenate([x_train_fismo,x_train_bow,x_train_sharma, x_train_still]) 
x_test = np.concatenate([x_test_fismo,x_test_bow,x_test_sharma, x_test_still]) 
y_train = np.concatenate([y_train_fismo,y_train_bow,y_train_sharma, y_train_still])
y_test = np.concatenate([y_test_fismo,y_test_bow,y_test_sharma, y_test_still])

print('x_train: ' + str(np.shape(x_train)))
print('x_test: '+ str(np.shape(x_test)))
print('y_train: ' + str(np.shape(y_train)))
print('y_test: ' + str(np.shape(y_test)))

print('dataset shape check clear~!! =====================================')

load input data clear~!! =========================================
load label data clear~!! =========================================
load dataset clear~!! ============================================
x_train_fismo: (3390, 224, 224, 3)
x_test_fismo: (283, 224, 224, 3)
y_train_fismo: (3390, 2)
y_test_fismo: (283, 2)
x_train_bow: (1083, 224, 224, 3)
x_test_bow: (91, 224, 224, 3)
y_train_bow: (1083, 2)
y_test_bow: (91, 2)
x_train_sharma: (3123, 224, 224, 3)
x_test_sharma: (261, 224, 224, 3)
y_train_sharma: (3123, 2)
y_test_sharma: (261, 2)
x_train: (12596, 224, 224, 3)
x_test: (1885, 224, 224, 3)
y_train: (12596, 2)
y_test: (1885, 2)
dataset shape check clear~!! =====================================


In [16]:
x_train_resize = []
x_test_resize = []

img_number, _, _, _ = np.shape(x_train)
for idx in range(img_number):
    x_train_resize.append( resize(x_train[idx], (64, 64)) )
    
img_number, _, _, _ = np.shape(x_test)
for idx in range(img_number):
    x_test_resize.append( resize(x_test[idx], (64, 64)) )

x_train_resize = np.array(x_train_resize)/255.
x_test_resize = np.array(x_test_resize)/255.

print("resized x_train data : " + str(np.shape(x_train_resize)) )
print("resized x_test data : " + str(np.shape(x_test_resize)) )

resized x_train data : (12596, 64, 64, 3)
resized x_test data : (1885, 64, 64, 3)


In [17]:
model.compile(loss = 'categorical_crossentropy', optimizer = opt_adam, metrics = ['accuracy'])

history = model.fit(x_train_resize, y_train,
              batch_size=batch_size,
              epochs=nb_epoch,
              shuffle=True,
              verbose=1,
              validation_data=(x_test_resize, y_test),
                #callbacks=[lrate])
                callbacks=[history, mcp_save])

predictions_valid = model.predict(x_test_resize, batch_size=batch_size, verbose=1)
# Combine 3 set of outputs using averaging
predictions_valid = sum(predictions_valid)/len(predictions_valid)


Train on 12596 samples, validate on 1885 samples
Epoch 1/50
12596/12596 [==============================] - 12s 944us/step - loss: 0.5955 - accuracy: 0.6944 - val_loss: 0.5436 - val_accuracy: 0.7708
Epoch 2/50
12596/12596 [==============================] - 10s 787us/step - loss: 0.5195 - accuracy: 0.7556 - val_loss: 0.5491 - val_accuracy: 0.7342
Epoch 3/50
12596/12596 [==============================] - 10s 810us/step - loss: 0.4769 - accuracy: 0.7849 - val_loss: 0.5062 - val_accuracy: 0.7698
Epoch 4/50
12596/12596 [==============================] - 11s 850us/step - loss: 0.4407 - accuracy: 0.8052 - val_loss: 0.4507 - val_accuracy: 0.8021
Epoch 5/50
12596/12596 [==============================] - 10s 808us/step - loss: 0.4106 - accuracy: 0.8195 - val_loss: 0.4195 - val_accuracy: 0.8186
Epoch 6/50
12596/12596 [==============================] - 10s 819us/step - loss: 0.3869 - accuracy: 0.8322 - val_loss: 0.3924 - val_accuracy: 0.8424
Epoch 7/50
12596/12596 [==============================] 

In [ ]:
# from here, test part work

In [8]:
#loaded_model = DCR_block_classification(img_rows, img_cols, channel)
loaded_model = model
loaded_model.load_weights('./ckpt_folder/firenet_still.hdf5')
opt_adam = keras.optimizers.Adam(lr=learning_rate)
loaded_model.compile(optimizer=opt_adam, loss='categorical_crossentropy', metrics=['acc'])

In [9]:
loaded_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 16)        448       
_________________________________________________________________
average_pooling2d_1 (Average (None, 32, 32, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        4640      
_________________________________________________________________
average_pooling2d_2 (Average (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 16, 32)        0   

In [10]:
# load dataset

with open ('./datafiles_aug/x_test_fismo', 'rb') as fp:
    x_test_fismo = pickle.load(fp, encoding='latin1')
with open ('./datafiles_aug/x_test_bow', 'rb') as fp:
    x_test_bow = pickle.load(fp, encoding='latin1')
with open ('./datafiles_aug/x_test_sharma', 'rb') as fp:
    x_test_sharma = pickle.load(fp, encoding='latin1')

print('load input data clear~!! =========================================')


with open ('./datafiles_aug/y_test_fismo', 'rb') as fp:
    y_test_fismo = pickle.load(fp, encoding='latin1')

with open ('./datafiles_aug/y_test_bow', 'rb') as fp:
    y_test_bow = pickle.load(fp, encoding='latin1')

with open ('./datafiles_aug/y_test_sharma', 'rb') as fp:
    y_test_sharma = pickle.load(fp, encoding='latin1')

# with open ('./datafiles_aug/x_train_still.pickle', 'rb') as fp:
#     x_train_still = pickle.load(fp, encoding='latin1')   

# with open ('./datafiles_aug/y_train_still.pickle', 'rb') as fp:
#     y_train_still = pickle.load(fp, encoding='latin1')   
    
with open ('./datafiles_aug/x_test_still.pickle', 'rb') as fp:
    x_test_still = pickle.load(fp, encoding='latin1') 

with open ('./datafiles_aug/y_test_still.pickle', 'rb') as fp:
    y_test_still = pickle.load(fp, encoding='latin1') 
    
    
    
print('load label data clear~!! =========================================')
print('load dataset clear~!! ============================================')
    

print('x_test_fismo: '+ str(np.shape(x_test_fismo)))

print('y_test_fismo: ' + str(np.shape(y_test_fismo)))
print('==================================================================')

print('x_test_bow: '+ str(np.shape(x_test_bow)))

print('y_test_bow: ' + str(np.shape(y_test_bow)))
print('==================================================================')

print('x_test_sharma: '+ str(np.shape(x_test_sharma)))

print('y_test_sharma: ' + str(np.shape(y_test_sharma)))

# print('x_train_still: '+ str(np.shape(x_train_still)))

# print('y_train_still: ' + str(np.shape(y_train_still)))

print('x_test_still: '+ str(np.shape(x_test_still)))

print('y_test_still: ' + str(np.shape(y_test_still)))


x_test = np.concatenate([x_test_fismo,x_test_bow,x_test_sharma,x_test_still]) / 255.
y_test = np.concatenate([y_test_fismo,y_test_bow,y_test_sharma,y_test_still])


# x_still = np.concatenate([x_train_still, x_test_still]) / 255.
# y_still = np.concatenate([y_train_still, y_test_still])

# print('x_train: ' + str(np.shape(x_train)))
print('x_test: '+ str(np.shape(x_test)))
# print('y_train: ' + str(np.shape(y_train)))
print('y_test: ' + str(np.shape(y_test)))

print('dataset shape check clear~!! =====================================')

load input data clear~!! =========================================
load label data clear~!! =========================================
load dataset clear~!! ============================================
x_test_fismo: (283, 224, 224, 3)
y_test_fismo: (283, 2)
x_test_bow: (91, 224, 224, 3)
y_test_bow: (91, 2)
x_test_sharma: (261, 224, 224, 3)
y_test_sharma: (261, 2)
x_test_still: (1250, 224, 224, 3)
y_test_still: (1250, 2)
x_test: (1885, 224, 224, 3)
y_test: (1885, 2)
dataset shape check clear~!! =====================================


In [11]:
fismo_resize = []
bow_resize = []
sharma_resize = []
still_resize =[]

img_number, _, _, _ = np.shape(x_test_fismo)
for idx in range(img_number):
    fismo_resize.append( resize(x_test_fismo[idx], (64, 64)) )
    
img_number, _, _, _ = np.shape(x_test_bow)
for idx in range(img_number):
    bow_resize.append( resize(x_test_bow[idx], (64, 64)) )
    
img_number, _, _, _ = np.shape(x_test_sharma)
for idx in range(img_number):
    sharma_resize.append( resize(x_test_sharma[idx], (64, 64)) )
    
img_number, _, _, _ = np.shape(x_test_still)
for idx in range(img_number):
    still_resize.append( resize(x_test_still[idx], (64, 64)) )

fismo_resize = np.array(fismo_resize) /255.
bow_resize = np.array(bow_resize) /255.
sharma_resize = np.array(sharma_resize) /255. 
still_resize = np.array(still_resize) /255. 

x_test_resize = np.concatenate([fismo_resize,bow_resize,sharma_resize, still_resize])

In [12]:
loss_and_metrics = loaded_model.evaluate(fismo_resize, np.array(y_test_fismo), batch_size=128)
print('')
print('loss_and_metrics_fismo : ' + str(loss_and_metrics))
fismo = round(loss_and_metrics[1],6)


loss_and_metrics = loaded_model.evaluate(bow_resize, np.array(y_test_bow), batch_size=128)
print('')
print('loss_and_metrics_bow : ' + str(loss_and_metrics))
bow = round(loss_and_metrics[1],6)


loss_and_metrics = loaded_model.evaluate(sharma_resize, np.array(y_test_sharma), batch_size=128)
print('')
print('loss_and_metrics_sharma : ' + str(loss_and_metrics))
sharma = round(loss_and_metrics[1],6)



loss_and_metrics = loaded_model.evaluate(still_resize, np.array(y_test_still), batch_size=128)
print('')
print('loss_and_metrics_still : ' + str(loss_and_metrics))
still = round(loss_and_metrics[1],6)


loss_and_metrics = loaded_model.evaluate(x_test_resize, np.array(y_test), batch_size=128)
print('')
print('loss_and_metrics_entire : ' + str(loss_and_metrics))
entire = round(loss_and_metrics[1],6)

print('')
print ('Accuracy')
print (entire, '&', fismo, '&',bow, '&',sharma, '&',still)



283/283 [==============================] - 1s 4ms/step

loss_and_metrics_fismo : [0.21747631003073165, 0.9434629082679749]
91/91 [==============================] - 0s 440us/step

loss_and_metrics_bow : [0.5775536894798279, 0.8461538553237915]
261/261 [==============================] - 0s 137us/step

loss_and_metrics_sharma : [0.16772477298803712, 0.938697338104248]
1250/1250 [==============================] - 0s 69us/step

loss_and_metrics_still : [0.3317923585653305, 0.8855999708175659]
1885/1885 [==============================] - 0s 70us/step

loss_and_metrics_entire : [0.3037770777861699, 0.8997347354888916]

Accuracy
0.899735 & 0.943463 & 0.846154 & 0.938697 & 0.8856


In [13]:
predictions_entire = loaded_model.predict(x_test_resize, batch_size=128, verbose=1)
predictions_fismo = loaded_model.predict(fismo_resize, batch_size=128, verbose=1)
predictions_bow = loaded_model.predict(bow_resize, batch_size=128, verbose=1)
predictions_sharma = loaded_model.predict(sharma_resize, batch_size=128, verbose=1)
predictions_still = loaded_model.predict(still_resize, batch_size=128, verbose=1)

1250/1250 [==============================] - 0s 33us/step


In [14]:
from keras import backend as K

In [15]:
def prediction_result(predictions_valid):
    result_list = []
    for i in range(len(predictions_valid)):
        if (predictions_valid[i][0] > predictions_valid[i][1]):
            result_list.append(0)
        else : result_list.append(1)
    return result_list

In [16]:
predictions_entire_tmp = prediction_result(predictions_entire)
y_test_tmp = prediction_result(y_test)
predictions_fismo_tmp = prediction_result(predictions_fismo)
y_test_fismo_tmp = prediction_result(y_test_fismo)
predictions_bow_tmp = prediction_result(predictions_bow)
y_test_bow_tmp = prediction_result(y_test_bow)
predictions_sharma_tmp = prediction_result(predictions_sharma)
y_test_sharma_tmp = prediction_result(y_test_sharma)
predictions_still_tmp = prediction_result(predictions_still)
y_test_still_tmp = prediction_result(y_test_still)

In [17]:
print('PRECISION')
entire= sm.precision_score(y_test_tmp, predictions_entire_tmp)
fismo = sm.precision_score(y_test_fismo_tmp, predictions_fismo_tmp)
bow = sm.precision_score(y_test_bow_tmp, predictions_bow_tmp)
sharma = sm.precision_score(y_test_sharma_tmp, predictions_sharma_tmp)
still = sm.precision_score(y_test_still_tmp, predictions_still_tmp)
print  (entire)
print  (fismo)
print  (bow)
print  (sharma)
# print  (sm.precision_score(y_train_still_tmp, predictions_still_train_tmp))
print  (still)

print('')
print (round(entire,6), '&', round(fismo,6), '&',round(bow,6), '&',round(sharma,6), '&',round(still,6))




entire= sm.recall_score(y_test_tmp, predictions_entire_tmp)
fismo = sm.recall_score(y_test_fismo_tmp, predictions_fismo_tmp)
bow = sm.recall_score(y_test_bow_tmp, predictions_bow_tmp)
sharma = sm.recall_score(y_test_sharma_tmp, predictions_sharma_tmp)
still = sm.recall_score(y_test_still_tmp, predictions_still_tmp)

print('')
print('RECALL')
print  (entire)
print  (fismo)
print  (bow)
print  (sharma)
# print  (sm.precision_score(y_train_still_tmp, predictions_still_train_tmp))
print  (still)

print('')
print (round(entire,6), '&', round(fismo,6), '&',round(bow,6), '&',round(sharma,6), '&',round(still,6))



entire= sm.f1_score(y_test_tmp, predictions_entire_tmp)
fismo = sm.f1_score(y_test_fismo_tmp, predictions_fismo_tmp)
bow = sm.f1_score(y_test_bow_tmp, predictions_bow_tmp)
sharma = sm.f1_score(y_test_sharma_tmp, predictions_sharma_tmp)
still = sm.f1_score(y_test_still_tmp, predictions_still_tmp)

print('')
print('F1SCORE')
print  (entire)
print  (fismo)
print  (bow)
print  (sharma)
# print  (sm.precision_score(y_train_still_tmp, predictions_still_train_tmp))
print  (still)

print('')
print (round(entire,6), '&', round(fismo,6), '&',round(bow,6), '&',round(sharma,6), '&',round(still,6))

PRECISION
0.8772504091653028
0.9692307692307692
0.9056603773584906
0.8571428571428571
0.8184713375796179

0.87725 & 0.969231 & 0.90566 & 0.857143 & 0.818471

RECALL
0.8246153846153846
0.949748743718593
0.8421052631578947
0.8235294117647058
0.749271137026239

0.824615 & 0.949749 & 0.842105 & 0.823529 & 0.749271

F1SCORE
0.8501189532117368
0.9593908629441623
0.8727272727272727
0.84
0.78234398782344

0.850119 & 0.959391 & 0.872727 & 0.84 & 0.782344


In [18]:
print ('all :')
print (sm.confusion_matrix(y_test_tmp, predictions_entire_tmp))
print ('fismo :')
print (sm.confusion_matrix(y_test_fismo_tmp, predictions_fismo_tmp))
print ('bow :')
print (sm.confusion_matrix(y_test_bow_tmp, predictions_bow_tmp))
print ('sharma :')
print (sm.confusion_matrix(y_test_sharma_tmp, predictions_sharma_tmp))
# print ('still_train :')
# print (sm.confusion_matrix(y_train_still_tmp, predictions_still_train_tmp))
print ('still :')
print (sm.confusion_matrix(y_test_still_tmp, predictions_still_tmp))

all :
[[1160   75]
 [ 114  536]]
fismo :
[[ 78   6]
 [ 10 189]]
bow :
[[29  5]
 [ 9 48]]
sharma :
[[203   7]
 [  9  42]]
still :
[[850  57]
 [ 86 257]]


In [19]:
tn, fp, fn, tp =  sm.confusion_matrix(y_test_tmp, predictions_entire_tmp).ravel()

entire_FAR = float(fp)/(tn+fp)
print("FAR value - all : %1.8f"%entire_FAR)

tn, fp, fn, tp =  sm.confusion_matrix(y_test_fismo_tmp, predictions_fismo_tmp).ravel()

fismo_FAR = float(fp)/(tn+fp)
print("FAR value - fismo : %1.8f"%fismo_FAR)

tn, fp, fn, tp =  sm.confusion_matrix(y_test_bow_tmp, predictions_bow_tmp).ravel()

bow_FAR = float(fp)/(tn+fp)
print("FAR value - bow : %1.8f"%bow_FAR)

tn, fp, fn, tp =  sm.confusion_matrix(y_test_sharma_tmp, predictions_sharma_tmp).ravel()
sharma_FAR = float(fp)/(tn+fp)
print("FAR value - sharma : %1.8f"%sharma_FAR)

tn, fp, fn, tp =  sm.confusion_matrix(y_test_still_tmp, predictions_still_tmp).ravel()
still_FAR = float(fp)/(tn+fp)
print("FAR value - still : %1.8f"%still_FAR)


print('')
print ('FAR')
print (round(entire_FAR*100,6), '&', round(fismo_FAR*100,6), '&',round(bow_FAR*100,6), '&',round(sharma_FAR*100,6), '&',round(still_FAR*100,6))

FAR value - all : 0.06072874
FAR value - fismo : 0.07142857
FAR value - bow : 0.14705882
FAR value - sharma : 0.03333333
FAR value - still : 0.06284454

FAR
6.072874 & 7.142857 & 14.705882 & 3.333333 & 6.284454
